In [15]:
import requests
import pandas as pd
import xml.etree.cElementTree as et

In [10]:
# Get data and write XML (URL should result from a query)
response = requests.get("https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=p53")

data = response.content.decode()

with open("my_xml.xml", 'w') as f:
    f.writelines(data)

In [23]:
# Parse XML and extract sensible tags
tree = et.parse("my_xml.xml")
root = tree.getroot()

IDs = []
for pmc in root.iter("pmcid"):
    IDs.append(pmc.text)

IDs

In [24]:
# get full text per paper
# process that
# NLP that with spacy, after loading language model
